In [1]:
# import necessary packages & libraries
import mdptoolbox
from itertools import product
import numpy as np

In [2]:
# create table containing all potential permutations of actions, fields, item colours and item placings
def state_table():
    
    # states can be constituted in the form of an array containing field position, action (store/restore) and colour
    options_actions = 2
    number_fields = 4
    options_colors = 3
    options_placings = 4

    # create array of potential state permutations (1.536 x 6 [4x fields, 1x action, 1x colour])
    permutations = np.ndarray(shape=(options_actions*options_colors*(options_placings**number_fields), number_fields+2))

    # create list of potential field states (iterate over the different placing options per each single field)
    field_states = list(product(np.arange(options_placings), repeat=number_fields))

    for field_states_count, field_state in enumerate(field_states):
        for actions_count in range(options_actions):
            for colours_count in range(options_colors):
                # Index states
                state_indices = options_actions * options_colors * field_states_count + actions_count * options_colors + colours_count
                
                # Perform colours count shifting
                permutations[state_indices, :] = *field_state, colours_count, actions_count

    return permutations

states = state_table()
states[:3]

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 2., 0.]])

In [3]:
def trans_prob(states):

    options_actions = 2
    number_fields = 4
    options_colors = 3
    options_placings = 4
    n_states = states.shape[0]

    # return array of given shape and type filled with zeros
    # number_fields + 1 due to start/stop field
    trans = np.zeros((number_fields + 1, n_states, n_states))

    for state1 in range(n_states):
        for field in range(number_fields):
            for state2 in range(n_states):
                first_st = states[state1]
                second_st = states[state2]

                number_action = first_st[-2]
                number_column = second_st[-1]

                # in this case: put/store object
                if number_action == 0:
                    if first_st[field] == 3 and second_st[field] == number_column:
                        trans[field, state1, state2] = 1
                    else:
                        trans[4, state1, state2] = 1
                # in this case: pick/restore object
                else:
                    if first_st[field] == number_column and second_st[field] == 3:
                        trans[field, state1, state2] = 1
                    else:
                        trans[4, state1, state2] = 1

            row_sum = np.sum(trans[field, state1, :])
            # values are subject to division assignment (divided by row sum) in order to perform normalization (prob. sum = 100%)
            if row_sum > 1:
                trans[field, state1, :] /= row_sum
            else:
                trans[field, state1, state1] = 1
            
        row_sum = np.sum(trans[4, state1, :])
        
        if row_sum > 1:
            trans[4, state1, :] /= row_sum
        else:
            trans[4, state1, state1] = 1
            
    return trans

trans = trans_prob(states)
trans[:1]

array([[[1.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.00260417, 0.00260417, 0.00260417, ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         1.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 1.        ]]])

In [5]:
# create a reward matrix that rewards shortest-possible distances taken by the agent with a bonus
def reward_matrix(states, trans):

    layout_fields = (2,2)
    options_actions = 2
    number_fields = 4
    options_colors = 3
    options_placings = 4

    # the 0th tuple element refers to the rows, while the 1st tuple refers to the columns of the layout
    col = np.arange(layout_fields[1])
    row = np.arange(layout_fields[0])

    # set up new array filles with ones across the fields
    dist = np.ones(layout_fields)

    # transpose distance arrays
    dist += col
    dist = (dist.T + row).T

    dist = dist.flatten()
    dist = np.append(dist, 7)

    results = []

    # 
    for action_number, sparse_matrix in enumerate(trans):
        indices = np.where(np.logical_and(sparse_matrix > 0, sparse_matrix < 1))
        reward_ratings = np.zeros(sparse_matrix.shape)
        reward_ratings[indices] = 10 - dist[action_number]

        results.append(reward_ratings)

    return results

rewards = reward_matrix(states, trans)
rewards[:3]

[array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [9., 9., 9., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [8., 8., 8., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [8., 8., 8., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])]

In [8]:
# Perform MDP to find optimal (bonus-maximising) policy of the agent

policy_best = mdptoolbox.mdp.ValueIteration(transitions=trans, reward=rewards, discount=0.95)
policy_best.setVerbose()
results = policy_best.run()
print(policy_best.policy)

  Iteration		V-variation
    1		  6.0
    2		  0.29687500000002487
    3		  0.10701029459635336
    4		  0.0633163226047877
    5		  0.031968815284610486
    6		  0.016004446243051973
    7		  0.00795781280146457
    8		  0.003954818209884081
    9		  0.001964754670105151
    10		  0.0009760481685603395
    11		  0.0004848692046479641
Iterating stopped, epsilon-optimal policy found.
(4, 0, 0, 4, 0, 0, 4, 0, 0, 4, 0, 0, 4, 0, 0, 4, 0, 0, 3, 0, 0, 3, 0, 0, 4, 0, 0, 4, 0, 0, 4, 0, 0, 4, 0, 0, 4, 0, 0, 4, 0, 0, 3, 0, 0, 3, 0, 0, 4, 0, 0, 4, 0, 0, 4, 0, 0, 4, 0, 0, 4, 0, 0, 4, 0, 0, 3, 0, 0, 3, 0, 0, 2, 0, 0, 2, 0, 0, 2, 0, 0, 2, 0, 0, 2, 0, 0, 2, 0, 0, 2, 0, 0, 2, 0, 0, 4, 0, 0, 4, 0, 0, 4, 0, 0, 4, 0, 0, 4, 0, 0, 4, 0, 0, 3, 0, 0, 3, 0, 0, 4, 0, 0, 4, 0, 0, 4, 0, 0, 4, 0, 0, 4, 0, 0, 4, 0, 0, 3, 0, 0, 3, 0, 0, 4, 0, 0, 4, 0, 0, 4, 0, 0, 4, 0, 0, 4, 0, 0, 4, 0, 0, 3, 0, 0, 3, 0, 0, 2, 0, 0, 2, 0, 0, 2, 0, 0, 2, 0, 0, 2, 0, 0, 2, 0, 0, 2, 0, 0, 2, 0, 0, 4, 0, 0, 4, 0, 0, 4, 0, 0, 4, 0, 0, 4